# USING ROI 

In [4]:
from pathlib import Path
import nibabel as nib
import numpy as np
import h5py



In [5]:
datapath = "/home/jovyan/shared/NSD/"
#lh_mtl_atlas = nib.load("./nsd-data/lh.MTL.nii.gz").get_fdata()
#rh_mtl_atlas = nib.load("./nsd-data/rh.MTL.nii.gz").get_fdata()
lh_mtl_atlas = nib.load("./nsd-data/roi/lh.MTL.nii.gz").get_fdata()
rh_mtl_atlas = nib.load("./nsd-data/roi/rh.MTL.nii.gz").get_fdata()
#lh_mtl_atlas = nib.load(datapath+"nsddata/ppdata/subj01/anat/roi/lh.MTL.nii.gz").get_fdata()
#rh_mtl_atlas = nib.load(datapath+"nsddata/ppdata/subj01/anat/roi/rh.MTL.nii.gz").get_fdata()
print(lh_mtl_atlas.shape)
print(type(lh_mtl_atlas))
print(np.max(lh_mtl_atlas), np.min(lh_mtl_atlas)) # maximum value supposed to be 10 ! 

(81, 104, 83)
<class 'numpy.ndarray'>
10.0 0.0


In [ ]:
# COrrect version with ROI 


for curr_region_idx in range(1, 10+1):
    lh_idx = np.where(lh_mtl_atlas==curr_region_idx) # fetch the index, but not adapted in size!
    rh_idx = np.where(rh_mtl_atlas==curr_region_idx)
    
    #print(len(lh_idx))
    #i=0
    #print(lh_idx[2][i], lh_idx[1][i], lh_idx[0][i])
    
    
    #print(len(lh_idx[0]))
    #print([lh_idx[1][i] for i in range(len(lh_idx[0]))]) PROBLEM: Index (118) out of range for (0-82) 
    #(lh_idx values are bigger than the size to the f["betas"] array !!!
    #print([ lh_idx[2][i], lh_idx[1][i], lh_idx[0][i] for i in range(len(lh_idx[0])])

    all_sessions = []
    #for sess in range(1, 40+1):
    for sess in range(1, 37+1): # in the shared folder, only 37 sessions
    #for sess in range(1, 3+1):
        file_path = Path(f"/home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session{int(sess):02d}.hdf5")
        if file_path.is_file():
            print(f"FOUND {file_path}")
            curr_session = []
            with h5py.File(file_path, "r") as f:
                print(f["betas"][0, :].shape)
                curr_session += [f["betas"][:, lh_idx[2][i], lh_idx[1][i], lh_idx[0][i]] for i in range(len(lh_idx[0]))]
                curr_session += [f["betas"][:, rh_idx[2][i], rh_idx[1][i], rh_idx[0][i]] for i in range(len(rh_idx[0]))]
                curr_session = np.array(curr_session)
                print(curr_session.shape)
            all_sessions.append(curr_session)

    all_sessions = np.concatenate(all_sessions, axis=1) # voxels, 750*sessions
    np.save(f"nsd-data/MTL-{curr_region_idx}_full.npy", all_sessions.T)

FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session01.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session02.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session03.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session04.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session05.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session06.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session07.hdf5
(83, 104, 81)
(345, 750)
FOUND /home/j

In [ ]:
# make sure hdf5 is the same as nifti
betas_nifti = nib.load("/home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session01.nii.gz")
betas_hdf5 = h5py.File("/home/jovyan/shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session01.hdf5", "r")

In [ ]:


np.allclose(betas_nifti.slicer[...,0].get_fdata(), np.swapaxes(betas_hdf5["betas"][0, :, :, :], 0, 2))

